In [ ]:
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import numpy as np
from datetime import datetime
import random
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import os
from math import sqrt


In [ ]:
dataset=pd.read_csv('train.csv')
dataset.head()

In [ ]:
dataset.info()

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_features = list(dataset.select_dtypes(include=numerics).columns) 
categorical_features = list(dataset.select_dtypes(include=['object']).columns) 
#numerical_features=numerical_features[1:-1]
print(numerical_features)
print(categorical_features)
dataset=dataset[numerical_features]
print(dataset.head())

In [ ]:
y = dataset.iloc[:, -1].values
data= dataset.drop(['Id', 'SalePrice'], axis=1)
display(data)
X = data.iloc[:, :].values


print(X.shape)
print(X)
print(y)

In [ ]:
split = int(X.shape[0] * 0.75)

x_train = X[:split,:]
y_train = y[:split]

x_test = X[split:,:]
y_test = y[split:]

val_split = int(x_test.shape[0] * 0.50)
x_val = x_test[:val_split,:]
y_val = y_test[:val_split]

x_test = x_test[val_split:,:]
y_test = y_test[val_split:]

print('Training set size:', x_train.shape[0])
print('Test set size:', x_test.shape[0])
print('Validation set size:', x_val.shape[0])

In [ ]:
def rmse(pred, y):
    return sqrt(mean_squared_error(pred,y))

In [ ]:
def find_split(x, y, n_features):
    """Given a dataset and its target values, this finds the optimal combination
    of feature and split point that gives the maximum information gain."""
    
   
    # Best thus far, initialised to a dud that will be replaced immediately...
    best = {'minimize_variance' : np.inf}
    
    # Random subspace
    features = np.random.randint(x.shape[1], size =  n_features)
 
    # Loop every possible split of every dimension...
    for feature in features:
        for split in np.unique(x[:,feature]):
            #print(split)
            left_indices = []
            right_indices = []
            
            for index, row in enumerate(x):
                if row[feature] <= split:
                    left_indices.append(index)
                else:
                    right_indices.append(index)
                    
            n = float(len(left_indices) + len(right_indices))
            nl = float(len(left_indices))
            nr = float(len(right_indices))
            var_r = 0.0
            var_l = 0.0
            if len(y[left_indices]) > 0:
                var_l = np.var(y[left_indices])
            
            if len(y[right_indices]) > 0:
                var_r = np.var(y[right_indices])
            
            minimize_variance = float(nl/n) * var_l + float(nr/n) * var_r
            
            if minimize_variance < best['minimize_variance']:
                #print(minimize_variance)
                best = {'feature' : feature,
                        'split' : split,
                        'minimize_variance' : minimize_variance, 
                        'left_indices' : left_indices,
                        'right_indices' : right_indices}
    return best
        

The function find_split() allows us to find the optimal feature and the best value to split the data into two chunks (on its own it is the decision stump algorithm). Applying this to the original data set splits it into two new data sets. We can then repeat this on both of the new data sets to get four data sets, and so on. This recursion builds a decision tree. It needs a stopping condition, to prevent it dividing the data forever, here we will use two:

Maximum depth: The tree is limited to be no deeper than a provided limit.
Perfection: If a node contains only one class then it does not make sense to split it further.
We provide the function build_tree(x, y, max_depth) below to construct a tree. The inputs are:

The data matrix of features, x in R^None. n is the number of data points and d is the feature dimensionality.
y, a column vector of size n containing the target value for each data point in x.
The maximum depth of the tree, max_depth.
The output of this function is a dictionary. If it has generated a leaf node then the keys are:

'leaf' : True
'mean' : Mean of the values of y to assign to exemplars that land here.
If it has generated a split node then the keys are:

'leaf' : False
'feature': The feature to apply the split to.
'split': The split to test the exemplars feature with.
'minimize_variance': The variance minimized of this split.
'left' : The left subtree, for exemplars where x[feature_index]<=split
'right' : The right subtree, for exemplars where x[feature_index]>split
Note how this structure is compatable with the one returned by find_split() above.

In [ ]:
def build_tree(x, y, max_depth = np.inf, n_features=6):
    # Check if either of the stopping conditions have been reached. If so generate a leaf node...
    move = find_split(x, y, n_features)
    if max_depth==1 or len(move['left_indices']) == 0  or len(move['right_indices']) == 0:
        # Generate a leaf node...
        return {'leaf' : True, 'mean' : np.mean(y)}
    
    else:
        left = build_tree(x[move['left_indices'],:], y[move['left_indices']], max_depth - 1, n_features)
        right = build_tree(x[move['right_indices'],:], y[move['right_indices']], max_depth - 1, n_features)
        
        return {'leaf' : False,
                'feature' : move['feature'],
                'split' : move['split'],
                'minimize_variance' : move['minimize_variance'],
                'left' : left,
                'right' : right}

After building the tree we should be able to predict the mean of a sample. We do that by propagating the sample through the tree, i.e. we check all the splitting conditions until the sample falls in a leaf node, in which case the mean of the leaf node is attributed to the sample.

We further generalize the prediction function above to the case where we have a data matrix R^None representing many data points. the function predict(tree, samples) bellow takes as input the constructed tree and a data array then returns an array containing the predictions for all the samples in our input data array.



In [ ]:
def predict(tree, samples):
    """Predicts mean for every entry of a data matrix."""
    ret = np.empty(samples.shape[0], dtype=float)
    ret.fill(0.0)
    indices = np.arange(samples.shape[0])
    
    def tranverse(node, indices):
        nonlocal samples
        nonlocal ret
        
        if node['leaf']:
            ret[indices] = node['mean']
        
        else:
            going_left = samples[indices, node['feature']] <= node['split']
            left_indices = indices[going_left]
            right_indices = indices[np.logical_not(going_left)]
            
            if left_indices.shape[0] > 0:
                tranverse(node['left'], left_indices)
                
            if right_indices.shape[0] > 0:
                tranverse(node['right'], right_indices)
    
    tranverse(tree, indices)
    return ret

In [ ]:
def forest_predict(trees, dataset):
    predictions = np.zeros((dataset.shape[0], len(trees)), dtype=float)

    for i in range (len(trees)):
        #print('------------- Tree -----------------------')
        prediction = predict(trees[i], dataset)
        predictions[:, i] = prediction
    
    return np.mean(predictions, axis=1)

In [ ]:
def bootstrap(n):
    draws = np.random.choice(len(n), size =  len(n))
    return draws

In [ ]:
#best_num_trees = 5 # Hard coded to due to fixed computational budget
#best_feature_number = 6 # Hardcoded to sqrt of number of feautures due to fixed computational budget

def train(x, y, max_depth, n_trees, n_features):
    current_directory = os.getcwd()
    final_directory = os.path.join(current_directory, r'pickles')
    if not os.path.exists(final_directory):
        os.makedirs(final_directory)
    trees = []
    for i in range(n_trees):
        filename = 'tree_' + str(len(x)) + '_' + str(max_depth) + '_' + str(n_trees) + '_' + str(n_features) + '_' + str(i)
        for root, dirs, files in os.walk(final_directory):
            if filename in files:
                filename = os.path.join(root, filename)
                pf = open(filename,"rb")
                tree = pickle.load(pf)
                print("Found tree in: " + filename)
            else:
                bootstrap_draws = bootstrap(x)
                #bootstrap_draws = np.arange(len(x))
                #print(bootstrap_draws)
                tree = build_tree(x[bootstrap_draws], y[bootstrap_draws], max_depth, n_features)
                filehandler = open(os.path.join(final_directory, filename), 'wb')
                pickle.dump(tree, filehandler)
                print("storing: " + filename)
        trees.append(tree)
    print(f'----- trees:{len(trees)} Depth:{max_depth} Features:{n_features} ------')
    return trees

# Train
print(datetime.now())
temp_forest = train(x_train, y_train, np.inf, 10, 6)
print(datetime.now())
y_train_pred = forest_predict(temp_forest, x_train)
train_mse = rmse(y_train,y_train_pred)
print("train_mse: {:.2f}".format(train_mse))

r2_score(y_train, y_train_pred)


In [ ]:
def find_best_hyperparameters(x_t, y_t, x_val, y_val):
    best_val_rmse = float("inf")
    best_max_depth = 0
    best_num_trees = 0
    best_num_features = 0
    depths = [5, 10, 20, 30, 40, 60,100]
    num_trees = [5, 10, 20, 30, 40, 50]
    num_features = [15, 30, 60, 120, 240]
    rmse = []
    for num_tree in num_trees:
        for num_feature in num_features:
            for depth in depths:
                temp_forest = train(x_t, y_t, depth, num_tree, num_feature)
                y_val_pred = forest_predict(temp_forest, x_val)
                d_val = np.subtract(y_val, y_val_pred)
                temp_val_rmse = np.sqrt(np.mean(d_val**2))
                rmse.append(temp_val_rmse)
                print("val_rmse: {:.2f}".format(temp_val_rmse))
                if temp_val_rmse < best_val_rmse:
                    best_max_depth = depth
                    best_num_trees = num_tree
                    best_num_features = num_feature
                    best_val_rmse = temp_val_rmse
    print("best_max_depth", best_max_depth)
    print("best_num_trees", best_num_trees)
    print("best_num_features", best_num_features)
    return best_max_depth, best_num_trees, best_num_features, best_val_rmse

#print(datetime.now())
#best_max_depth, best_num_trees, best_num_features, best_val_rmse = find_best_hyperparameters(x_train, y_train, x_val, y_val)
#print('The best max_depth is {} and the corresponding val RMSE is {:.2f} .'.format(best_max_depth, best_val_rmse))
#print(datetime.now())

In [ ]:
best_max_depth = 0
best_num_trees = 0
best_num_features = 0
#Test
print(datetime.now())
best_forest = train(x_train, y_train, np.inf, 10, 6)
y_train_pred = forest_predict(best_forest, x_train)
train_mse = np.square(np.subtract(y_train,y_train_pred)).mean()
print('Train MSE score : {:.2f}'.format(train_mse))
y_test_pred = forest_predict(best_forest, x_test)
test_mse = np.square(np.subtract(y_test,y_test_pred)).mean()
print('Test MSE score : {:.2f}'.format(test_mse))
print(datetime.now())

r2_score(y_train, y_train_pred)

In [ ]:
#submit to kaggle

test_data=pd.read_csv('test.csv')
test_data.head()

In [ ]:
dataset=test_data[numerical_features[:-1]]

data= dataset.drop(['Id'], axis=1)
display(data)
x_sub = data.iloc[:, :].values
print(dataset.head())
y_sub_pred = forest_predict(best_forest, x_sub)

print(y_sub_pred)

submission = pd.DataFrame({'Id': dataset.Id, 'SalePrice': y_sub_pred})
submission.to_csv('submission.csv', index=False)